In [2]:
import sympy as sp
from functools import cmp_to_key
from sympy.parsing.sympy_parser import parse_expr

def parse_expressions(str_expressions, variable):
    """
    Konvertiert eine Liste von Strings sicher in SymPy-Ausdrücke.
    Diese Version ist korrigiert und robuster.
    """
    parsed = []
    for s in str_expressions:
        # Korrekte Initialisierung der Variable
        s_parsed = s 
        
        # Spezifische Ersetzungen, um korrekte SymPy-Syntax zu erzeugen
        # Wichtig: Wir ersetzen spezifische Funktionsaufrufe
        if 'log2(n)' in s_parsed:
            s_parsed = s_parsed.replace('log2(n)', 'log(n, 2)')
        if 'log10(n)' in s_parsed:
            s_parsed = s_parsed.replace('log10(n)', 'log(n, 10)')
        if 'ln(n)' in s_parsed:
            s_parsed = s_parsed.replace('ln(n)', 'log(n)')
        
        # Ersetze n.log(n) durch n*log(n) für SymPy
        if 'n.log(n)' in s_parsed:
            s_parsed = s_parsed.replace('n.log(n)', 'n*log(n)')
        
        # Nutze parse_expr für robustes Parsen
        parsed.append(parse_expr(s_parsed, local_dict={'n': variable, 'log': sp.log}))
        
    return parsed

def compare_growth(f1, f2, variable):
    """
    Vergleicht die asymptotische Wachstumsrate von zwei Funktionen f1 und f2.
    """
    if f1 == f2:
        return 0
    
    try:
        limit = sp.limit(f1 / f2, variable, sp.oo)
        if limit == 0:
            return -1
        elif limit.is_infinite:
            return 1
        else:
            return 0
    except Exception:
        return 0

def sort_complexity_classes(str_expressions):
    """
    Nimmt eine Liste von Funktionen (als Strings), sortiert sie nach
    Komplexitätsklassen und gibt das Ergebnis formatiert aus.
    """
    n = sp.Symbol('n', positive=True)
    
    expressions = parse_expressions(str_expressions, n)
    
    sorted_expr = sorted(expressions, key=cmp_to_key(lambda f1, f2: compare_growth(f1, f2, n)))
    
    if not sorted_expr:
        return ""
        
    grouped_classes = [[sorted_expr[0]]]
    for i in range(1, len(sorted_expr)):
        if compare_growth(sorted_expr[i], grouped_classes[-1][0], n) == 0:
            grouped_classes[-1].append(sorted_expr[i])
        else:
            grouped_classes.append([sorted_expr[i]])
            
    output_parts = []
    for group in grouped_classes:
        # sp.sstr() wird für eine saubere String-Darstellung verwendet
        group_str = " = ".join([f"O({sp.sstr(expr, full_prec=False)})" for expr in group])
        output_parts.append(group_str)
        
    return " ⊆ ".join(output_parts)

# =============================================================
# --- HAUPTPROGRAMM ---
# =============================================================
if __name__ == "__main__":
    
    funktionen_als_strings = [
        'n',
        'n + 1000',
        'n**2',
        'n**3',
        'n**2 + n',
        'log10(n)',
        'ln(n)',
        'log2(n)',
        'n + log(n)',
        'n.log(n)',
        '2**n',
        '3**n',
        'n**n'
    ]
    
    print("Ungeordnete Liste der Komplexitätsklassen:")
    for f in funktionen_als_strings:
        print(f"  O({f})")
        
    print("\n" + "="*60)
    print("Automatisch sortierte und gruppierte Lösung:")
    print("="*60)
    
    sortierte_loesung = sort_complexity_classes(funktionen_als_strings)
    print(sortierte_loesung)

Ungeordnete Liste der Komplexitätsklassen:
  O(n)
  O(n + 1000)
  O(n**2)
  O(n**3)
  O(n**2 + n)
  O(log10(n))
  O(ln(n))
  O(log2(n))
  O(n + log(n))
  O(n.log(n))
  O(2**n)
  O(3**n)
  O(n**n)

Automatisch sortierte und gruppierte Lösung:
O(log(n)/log(10)) = O(log(n)) = O(log(n)/log(2)) ⊆ O(n) = O(n + 1000) = O(n + log(n)) ⊆ O(n*log(n)) ⊆ O(n**2) = O(n**2 + n) ⊆ O(n**3) ⊆ O(2**n) ⊆ O(3**n) ⊆ O(n**n)
